In [112]:
# %load heart_ava.py
import pandas as pd
#import math
#import scipy as sp
import numpy as np
#from random import shuffle
#from scipy.stats import zscore
from sklearn.model_selection import train_test_split
#from sklearn.multiclass import OneVsRestClassifier
#from sklearn.svm import LinearSVC, SVC
from sklearn import preprocessing
#from sklearn.externals import joblib
from sklearn.metrics import confusion_matrix
#from os import listdir
#from os.path import isfile, join
from tqdm import tqdm

def filterDataset(data):
    invalidIndexes = []
    for row in data.itertuples():
        if '?' in row:
            invalidIndexes.append(row[0])
    new_data = data.drop(data.index[invalidIndexes])
    cols = list(new_data.columns)
    new_data[cols] = new_data[cols].astype('float32')
    return new_data

def train_and_test():
    #train = False
    #print('Reading data....')
    data = pd.read_csv('processed.cleveland.data', names=['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','num'], header=None)
    #print('Removing invalid values from data....')
    data = filterDataset(data)
    data = data[data['num']!=0]
    #data = data[data['num']!=4]
    ## split data into train and test
    cols = list(data.columns)
    cols.remove('num')
    target = data['num'].copy()
    input_data = data[cols].copy()
    dTrain, dTest, targetTrain, targetTest = train_test_split(input_data, target, test_size=0.50, stratify=target)
    train_index = dTrain.index
    scaler = preprocessing.StandardScaler().fit(dTrain)
    dTrain = scaler.transform(dTrain)
    cols = [i for i in range(dTrain.shape[1])]
    dTrain = pd.DataFrame(dTrain, index=train_index, columns=cols)
    labels = list(target.unique().astype(int))
    labels.sort()
    classes = labels.copy()
    votes = np.zeros((dTest.shape[0], max(classes)+1), dtype=np.int)
    clfs = []
    while len(classes) > 1:
        current_class = classes.pop(0)
        for c in classes:
            d = pd.concat([dTrain,targetTrain], axis=1, ignore_index=True)
            d = d.reset_index(drop=True)
            current_group = pd.concat([d[d[13]==current_class], d[d[13]==c]], axis=0, ignore_index=False)
            y = current_group[13].copy()
            cols = list(current_group.columns)
            cols.remove(13)
            x = current_group[cols].copy()
            #print('Training model for classes ', current_class, 'and ', c, '...')
            clf = SVC(kernel='linear').fit(x, y)
            clfs.append(clf)
    for clf in clfs:
        dTest = scaler.transform(dTest)
        cols = [i for i in range(dTest.shape[1])]
        dTest = pd.DataFrame(dTest, columns=cols)
        clf_pred = clf.predict(dTest)
        for i in range(clf_pred.shape[0]):
            votes[np.int(dTest.iloc[[i]].index[0]), np.int(clf_pred[i])] += 1
    pred = []
    for i in range(votes.shape[0]):
        #pred.append(np.argmax(votes[i,:]))
        pred.append(np.random.choice(np.where(votes[i,:] == votes[i,:].max())[0]))
    
    cm = confusion_matrix(targetTest, pred, labels=labels)
    np.savetxt("confusion_matrix_ava.csv", cm, delimiter=",")
    score = np.sum(np.diagonal(cm))/np.sum(cm)
    #print('score: ', score)
    return(score)
    

#if __name__ == "__main__":
    #main()


In [114]:
scores = []
for i in tqdm(range(100)):
    scores.append(train_and_test())
    
scores = np.asarray(scores)
print(scores.mean())
print(scores.std())

100%|██████████| 100/100 [00:16<00:00,  6.32it/s]

0.335797101449
0.0754195713408
